# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Bluff,99,NZ,1619818438,93,-46.6000,168.3333,44.01,4.00
1,1,Tasiilaq,41,GL,1619818438,86,65.6145,-37.6368,30.20,5.75
2,2,Souillac,40,MU,1619818439,83,-20.5167,57.5167,77.00,11.50
3,3,Punta Arenas,90,CL,1619818359,93,-53.1500,-70.9167,42.80,6.91
4,4,Grand Gaube,24,MU,1619818245,79,-20.0064,57.6608,78.01,7.00
5,5,Tuatapere,100,NZ,1619818164,93,-46.1333,167.6833,43.00,4.00
6,6,Saldanha,0,ZA,1619818440,67,-33.0117,17.9442,55.40,9.22
7,7,Rikitea,88,PF,1619818441,77,-23.1203,-134.9692,78.80,20.67
8,8,Hasaki,40,JP,1619818442,94,35.7333,140.8333,55.40,3.44
9,9,Shingū,53,JP,1619818442,57,33.7333,135.9833,59.34,2.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [5]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
possible_vacay = weather_file.loc[(weather_file["Wind Speed"] <= 10) & (weather_file["Cloudiness"] == 0) & \
                                   (weather_file["Max Temp"] >= 70) & (weather_file["Max Temp"] <= 80)].dropna()

possible_vacay

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
11,11,Buraidah,0,SA,1619818243,27,26.326,43.975,77.0,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
possible_hotels = possible_vacay.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [8]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
possible_hotels

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.3578299,
                    "lng": 43.9678808
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.35929347989272,
                        "lng": 43.96972462989272
                    },
                    "southwest": {
                        "lat": 26.35659382010728,
                        "lng": 43.96702497010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "M\u00f6venpick Hotel Qassim",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1371,
                    "html_attributions": [
                   

,City,Country,Lat,Lng,Hotel Name
11,Buraidah,SA,26.326,43.975,Mövenpick Hotel Qassim


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

hfigure.add_layer(markers)

# Display Map
hfigure


Figure(layout=FigureLayout(height='420px'))